In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("todos.csv")

In [ ]:
df.shape

(621, 9)

In [ ]:
df.head()

,Unnamed: 0,Text,Created at,target,Likes,Retweets,lang,geo,Hashtags
0,0,RT @SEC_cl: ⚠️Si eres una o uno de los afectad...,2022-07-03 6:39:34,1,0,1,es,NaN,"['CortedeLuz', 'Recoleta']"
1,1,#OtraVez!!! #CorteDeLuz #HastaCuando ya está r...,2022-07-03 4:25:11,1,0,0,es,Santiago de Chile,"['OtraVez', 'CorteDeLuz', 'HastaCuando', 'Solu..."
2,2,@CGE_Clientes qué pasa en Puente Alto qué sé c...,2022-07-02 23:06:08,1,0,0,es,Chile,"['CorteDeEnergia', 'CorteDeLuz']"
3,3,@saesa nuevamente #CorteDeLuz en sector Lingue...,2022-07-02 17:52:36,1,0,0,es,NaN,"['CorteDeLuz', 'mariquina']"
4,4,Atención: Edemsa cortará la luz en un departam...,2022-07-02 11:25:57,1,0,0,es,Mendoza,"['CorteDeLuz', 'Mendoza']"


In [ ]:
print((df.target== 1).sum())#CORTE DE SUMINISTRO
print((df.target== 0).sum())#NO CORTE DE SUMINISTRO

311
310


In [ ]:
#Preprocessing
import re
import string

def remove_URL(text):
  url=re.compile(r"https?://\S+|www\.\S+")
  return url.sub(r"", text)

#https://stackoverflow.com/questions/34293875/how-to-remove

def remove_punct(text):
  translator = str.maketrans("", "",string.punctuation)
  return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")

for t in df.Text:
  matches = pattern.findall(t)
  for match in matches:
    print(t)
    print(match)
    print(pattern.sub(r"", t))
  if len(matches) > 0:
    break

RT @SEC_cl: ⚠️Si eres una o uno de los afectados por el #CortedeLuz en #Recoleta haz tu reclamo en la empresa o en 👉https://t.co/MYKZoN5lBp
t
RT @SEC_cl: ⚠️Si eres una o uno de los afectados por el #CortedeLuz en #Recoleta haz tu reclamo en la empresa o en 👉


In [ ]:
df["text"] = df.Text.map(remove_URL) #map(lambdax:remove_URL(x))
df["text"] = df.Text.map(remove_punct)

In [ ]:
#remove stopwords
#pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Stop Words:Astop word isacommonly used word(such as"the","a","an","in")thatasearch engine
#has been programmed to ignore,both when indexing entries for searching and when retrieving them
#as the result ofasearch query.
stop = set(stopwords.words("spanish"))

#https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
  filtered_words=[word.lower() for word in text.split() if word.lower() not in stop]
  return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

In [ ]:
df["text"] = df.Text.map(remove_stopwords)

In [ ]:
df.Text

0      RT @SEC_cl: ⚠️Si eres una o uno de los afectad...
1      #OtraVez!!! #CorteDeLuz #HastaCuando ya está r...
2      @CGE_Clientes qué pasa en Puente Alto qué sé c...
3      @saesa nuevamente #CorteDeLuz en sector Lingue...
4      Atención: Edemsa cortará la luz en un departam...
                             ...                        
616    Mejorar el merchandising de las secciones del ...
617    RT @Gerard25111977: Por la Libertad en el Trab...
618                      @Danielredcarga Jajajaja 😂😂😂🤣🤣🤣
619    @MuniMixco_ @PNCdeGuatemala @MPguatemala : Jus...
620    @MuniMixco_ @PNCdeGuatemala @MPguatemala : Jus...
Name: Text, Length: 621, dtype: object

In [ ]:
from collections import Counter

#Count unique words
def counter_word(text_col):
  count = Counter()
  for text in text_col.values:
    for word in text.split():
      count[word] += 1
    return count
    
counter=counter_word(df.text)

In [ ]:
len(counter)

10

In [ ]:
counter

Counter({'#cortedeluz': 1,
         '#recoleta': 1,
         '@sec_cl:': 1,
         'afectados': 1,
         'empresa': 1,
         'haz': 1,
         'reclamo': 1,
         'rt': 1,
         '⚠️si': 1,
         '👉https://t.co/mykzon5lbp': 1})

In [ ]:
counter.most_common(5)

[('rt', 1), ('@sec_cl:', 1), ('⚠️si', 1), ('afectados', 1), ('#cortedeluz', 1)]

In [ ]:
num_unique_words = len(counter)

In [ ]:
#Split dataset into training and validation set
train_size=int(df.shape[0] * 0.8)

train_df = df[:train_size]
val_df = df[train_size:]

#split text and labels
train_sentences = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()

In [ ]:
train_sentences.shape, val_sentences.shape

((496,), (125,))

In [ ]:
#Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

#vectorizeatext corpus by turning each text intoasequence of integers
tokenizer=Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences)#fit only to training

In [ ]:
# cada palabra tiene su indice unico

word_index = tokenizer.word_index

In [ ]:
#word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [ ]:
print(train_sentences[10:35])
print(train_sequences[10:35])

['rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 '@enelclientescl hecho 12 horas resultado seguir #cortedeenergia #cortedeluz'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'corte luz oroño san juan, acá ven varias manzanas afectadas... onda @epeoficial ? #rosario #cortedeluz'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 '@chilquinta @chilquinta600 #cortedeluz varios sectores #quilpué'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tens

In [ ]:
#Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences
#Max number of words inasequence
max_length=201
train_padded=pad_sequences(train_sequences,maxlen=max_length,padding="post",truncating="post")
val_padded=pad_sequences(val_sequences,maxlen=max_length,padding="post",truncating="post")
train_padded.shape,val_padded.shape

((496, 201), (125, 201))

In [ ]:
train_padded[10]

array([6, 8, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)

In [ ]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…
[6, 8, 5]
[6 8 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
#Check reversing the indices

#flip(key,value)
reverse_word_index=dict([(idx,word)for(word,idx)in word_index.items()])

In [ ]:
reverse_word_index

{1: 'cortedeluz',
 2: 't',
 3: 'co',
 4: 'https',
 5: 'luz',
 6: 'rt',
 7: 'enelclientescl',
 8: 'oficialedesur',
 9: 'reclamo',
 10: '¡firma',
 11: 'edesur',
 12: 'enre',
 13: 'vez',
 14: 'petición',
 15: 'change',
 16: 'es',
 17: 'corte',
 18: 'cortedeenergia',
 19: 'false',
 20: 'menos',
 21: 'hace',
 22: 'vía',
 23: 'sector',
 24: 'sec',
 25: 'arg',
 26: 'energia',
 27: 'enel',
 28: 'almagro',
 29: 'defensoriacaba',
 30: 'cfkargentina',
 31: 'alferdez',
 32: 'gcba',
 33: 'ar',
 34: 'danielscioli',
 35: 'ñuñoa',
 36: 'presidente',
 37: 'horaciorlarreta',
 38: 'mas',
 39: 'cl',
 40: 'horas',
 41: 'energía',
 42: 'mes',
 43: 'servicio',
 44: '22065660313',
 45: 'inoperancia',
 46: '4797982',
 47: 'sinluz',
 48: 'si',
 49: 'laflorida',
 50: 'via',
 51: 'mykzon5lbp',
 52: 'san',
 53: 'empresa',
 54: 'hora',
 55: 'afectados',
 56: '⚠️si',
 57: 'haz',
 58: '👉https',
 59: 'día',
 60: 'pasa',
 61: 'media',
 62: 'tensión',
 63: 'cuadra',
 64: 'lindos',
 65: 'exploto',
 66: 'torre',
 67: 'seg

In [ ]:
def decode(sequence):
  return "".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [ ]:
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

[6, 8, 5]
rtoficialedesurluz


In [ ]:
#Create LSTM model
from tensorflow.keras import layers

#Embedding:https://www.tensorflow.org/tutorials/text/word_embeddings
#Turns positive integers(indexes)into dense vectors of fixed size.(other approach could be one-hot-encoding)
#Word embeddings give usaway to use an efficient,dense representation in which similar words have
#asimilar encoding.Importantly,you do not have to specify this encoding by hand.An embedding isa
#dense vector of floating point values(the length of the vector isaparameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length = max_length))

#The layer will take as input an integer matrix of size(batch,input_length),
#and the largest integer(i.e. word index)in the input should be no larger than num_words(vocabulary size).
#Now model.output_shape is(None,input_length,32),where`None is the batch dimension.

model.add(layers.LSTM(64,dropout=0.1))
model.add(layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 201, 32)           320       
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 25,217
Trainable params: 25,217
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer = optim, metrics = metrics)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_padded,train_labels,epochs=20,validation_data=(val_padded,val_labels),verbose=2)


Epoch 1/20
16/16 - 5s - loss: 0.6804 - accuracy: 0.5867 - val_loss: 0.9804 - val_accuracy: 0.0000e+00 - 5s/epoch - 290ms/step
Epoch 2/20
16/16 - 2s - loss: 0.6625 - accuracy: 0.6270 - val_loss: 0.9248 - val_accuracy: 0.0000e+00 - 2s/epoch - 110ms/step
Epoch 3/20
16/16 - 2s - loss: 0.6647 - accuracy: 0.6270 - val_loss: 0.9550 - val_accuracy: 0.0000e+00 - 2s/epoch - 110ms/step
Epoch 4/20
16/16 - 2s - loss: 0.6615 - accuracy: 0.6270 - val_loss: 0.9108 - val_accuracy: 0.0000e+00 - 2s/epoch - 111ms/step
Epoch 5/20
16/16 - 2s - loss: 0.6619 - accuracy: 0.6270 - val_loss: 0.9890 - val_accuracy: 0.0000e+00 - 2s/epoch - 110ms/step
Epoch 6/20
16/16 - 2s - loss: 0.6619 - accuracy: 0.6270 - val_loss: 0.9564 - val_accuracy: 0.0000e+00 - 2s/epoch - 110ms/step
Epoch 7/20
16/16 - 2s - loss: 0.6607 - accuracy: 0.6270 - val_loss: 1.0009 - val_accuracy: 0.0000e+00 - 2s/epoch - 112ms/step
Epoch 8/20
16/16 - 2s - loss: 0.6608 - accuracy: 0.6270 - val_loss: 1.0073 - val_accuracy: 0.0000e+00 - 2s/epoch - 107

In [ ]:
predictions = model.predict(train_padded)
predictions = [ 1 if p > 0.5 else 0 for p in predictions]

In [ ]:
print(train_sentences[10:20])
print(train_labels[10:20])
print(predictions[10:20])

['rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 '@enelclientescl hecho 12 horas resultado seguir #cortedeenergia #cortedeluz'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'corte luz oroño san juan, acá ven varias manzanas afectadas... onda @epeoficial ? #rosario #cortedeluz'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 '@chilquinta @chilquinta600 #cortedeluz varios sectores #quilpué'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tensión cuadra seguramente quedaremos luz po…'
 'rt @bernalaqui: lindos @oficialedesur ! vez exploto torre media tens